In [1]:
import sqlparse
import networkx as nx
import re
import pandas as pd

In [3]:
sql_content = open("./01_02_Claims.sql").read()

FileNotFoundError: [Errno 2] No such file or directory: './01_02_Claims.sql'

In [2]:
sql_statements = sqlparse.split(sql_content)
for sql_statement in sql_statements:
    print(sqlparse.parse(sql_statement))
    query_lines = []
    parsed_statement = sqlparse.parse(sql_statement)[0]
    for sql_line in parsed_statement.value.split("\n"):
        if not sql_line.startswith("#"):
            query_lines.append(" ".join(sql_line.split()))
    single_line_query = " ".join(query_lines).replace("[\s]+", " ")
    print((parsed_statement.get_type().lower()))
    print(single_line_query)
    print("--")

NameError: name 'sql_content' is not defined

# Dependency Generation

In [3]:
cd ..

/Users/bcard/Library/CloudStorage/OneDrive-AmericanInstitutesforResearchintheBehavioralSciences/projects/patentsview/PatentsView-DB/reporting_database_generator


In [20]:
ls

00_Creation.sql               05_02_index.sql
01_01_Govt_Interest.sql       05_03_index.sql
01_02_Claims.sql              05_04_index.sql
01_03_ID_Mappings.sql         05_05_index.sql
01_04_Application.sql         05_06_index.sql
01_05_Wipo.sql                05_07_index.sql
02_Patent.sql                 05_08_index.sql
03_01_Location.sql            05_09_index.sql
03_02_Assignee.sql            05_10_index.sql
03_03_Inventor.sql            05_11_index.sql
03_04_Lawyer.sql              05_12_index.sql
03_05_Examiner.sql            06_Reporting_Tables.sql
03_06_Foreign_Priority.sql    07_half_join.sql
03_07_PCT.sql                 __init__.py
03_08_US_App_Citation.sql     __pycache__/
03_09_US_Patent_Citation.sql  database/
03_10_CPC.sql                 dependency_analysis/
03_11_IPCR.sql                final.csv
03_12_Nber.sql                other_miscellaneous_tasks/
03_13_uspc.sql                reporting_db_location_id.csv
04_Support.sql                table_levels.csv
05_01_index.sq

In [23]:
index_queries = []
import os
list1 = os.listdir()
for i in list1:
    print(i)
#     if not i[0] == '0':
    if 'index' in i:
        index_queries.append(i)

03_12_Nber.sql
03_09_US_Patent_Citation.sql
05_10_index.sql
database
05_04_index.sql
table_levels.csv
05_09_index.sql
05_02_index.sql
.DS_Store
01_05_Wipo.sql
03_01_Location.sql
02_Patent.sql
01_01_Govt_Interest.sql
dependency_analysis
00_Creation.sql
01_02_Claims.sql
05_05_index.sql
05_11_index.sql
07_half_join.sql
03_13_uspc.sql
03_06_Foreign_Priority.sql
05_08_index.sql
05_03_index.sql
reporting_db_location_id.csv
03_04_Lawyer.sql
__init__.py
03_08_US_App_Citation.sql
__pycache__
03_07_PCT.sql
03_10_CPC.sql
05_12_index.sql
01_04_Application.sql
05_06_index.sql
03_03_Inventor.sql
01_03_ID_Mappings.sql
06_Reporting_Tables.sql
other_miscellaneous_tasks
03_05_Examiner.sql
03_02_Assignee.sql
03_11_IPCR.sql
05_07_index.sql
04_Support.sql
final.csv
05_01_index.sql


In [24]:
index_queries

['05_10_index.sql',
 '05_04_index.sql',
 '05_09_index.sql',
 '05_02_index.sql',
 '05_05_index.sql',
 '05_11_index.sql',
 '05_08_index.sql',
 '05_03_index.sql',
 '05_12_index.sql',
 '05_06_index.sql',
 '05_07_index.sql',
 '05_01_index.sql']

In [25]:
len(index_queries)

12

In [282]:
list1.remove('03_01_Location.sql')

In [5]:
my_list = ['02_Patent.sql','01_03_ID_Mappings.sql','03_01_Location.sql', '03_03_Inventor.sql', '03_02_Assignee.sql']

In [6]:
all_insert_queries = []
for sql_file in my_list:
    all_sql_content = open(sql_file).read()
    sql_statements = sqlparse.split(all_sql_content)

In [7]:
sql_statements

["# BEGIN assignee\n\n##############################################################################################################################################\n\n#this is necessary because otherwise the subqueries wierdly have a different collation\nSET collation_connection = 'utf8mb4_unicode_ci';",
 'drop table if exists `{{params.reporting_database}}`.`temp_assignee_lastknown_location`;',
 'create table `{{params.reporting_database}}`.`temp_assignee_lastknown_location`\n(\n    `assignee_id`            varchar(64)  not null,\n    `location_id`            int unsigned null,\n    `persistent_location_id` varchar(128) null,\n    `city`                   varchar(128) null,\n    `state`                  varchar(20)  null,\n    `country`                varchar(10)  null,\n    `latitude`               float        null,\n    `longitude`              float        null,\n    primary key (`assignee_id`)\n)\n    engine = InnoDB;',
 "# Populate temp_assignee_lastknown_location table.  The g

In [40]:
all_insert_queries = []
for sql_file in index_queries:
    all_sql_content = open(sql_file).read()
    sql_statements = sqlparse.split(all_sql_content)
    for sql_statement in sql_statements:
        query_lines = []
        parsed_statement = sqlparse.parse(sql_statement)[0]
        for sql_line in parsed_statement.value.split("\n"):
            if not sql_line.startswith("#"):
                query_lines.append(" ".join(sql_line.split()))
        if len(query_lines) < 1:
            continue
        single_line_query = " ".join(query_lines).replace("[\s]+", " ").replace(
            "{{params.reporting_database}}", "PatentsView_20220630").replace(
                "{{params.raw_database}}", "patent")
        if parsed_statement.get_type().lower() == 'ddl':
            if single_line_query.startswith("DROP"):
                continue
#         if parsed_statement.get_type().lower() == 'insert':
        all_insert_queries.append([single_line_query,sql_file])

In [39]:
all_insert_queries

['alter table `PatentsView_20220630`.`location_uspc_mainclass` add index `ix_location_uspc_mainclass_num_patents` (`num_patents`);',
 'alter table `PatentsView_20220630`.`location_uspc_mainclass` add index `ix_location_uspc_mainclass_mainclass_id` (`mainclass_id`);',
 'alter table `PatentsView_20220630`.`location_uspc_mainclass` add index `ix_location_uspc_mainclass_location_id` (`location_id`);',
 'alter table `PatentsView_20220630`.`location_year` add index `ix_location_year_year` (`year`);',
 'alter table `PatentsView_20220630`.`location_year` add index `ix_location_year_location_id` (`location_id`);',
 'alter table `PatentsView_20220630`.`location_year` add index `ix_location_year_num_patents` (`num_patents`);',
 'alter table `PatentsView_20220630`.`location` add index `ix_location_county` (`county`);',
 'alter table `PatentsView_20220630`.`location` add index `ix_location_state_fips` (`state_fips`);',
 'alter table `PatentsView_20220630`.`location` add index `ix_location_county_fi

In [42]:
final_table_list

['`PatentsView_20220630`.`temp_patent_firstnamed_assignee`',
 '`PatentsView_20220630`.`temp_id_mapping_assignee`',
 '`PatentsView_20220630`.`temp_id_mapping_location`',
 '`PatentsView_20220630`.`temp_patent_firstnamed_inventor`',
 '`PatentsView_20220630`.`temp_id_mapping_inventor`',
 '`PatentsView_20220630`.`temp_id_mapping_location`',
 '`PatentsView_20220630`.`patent`',
 '`PatentsView_20220630`.`temp_patent_date`',
 '`PatentsView_20220630`.`temp_patent_firstnamed_assignee`',
 '`PatentsView_20220630`.`temp_patent_firstnamed_inventor`',
 '`PatentsView_20220630`.`temp_patent_aggregations`',
 '`PatentsView_20220630`.`temp_patent_earliest_application_date`',
 '`PatentsView_20220630`.`temp_id_mapping_location`',
 '`PatentsView_20220630`.`temp_location_num_assignees`',
 '`PatentsView_20220630`.`temp_id_mapping_location`',
 '`PatentsView_20220630`.`temp_location_num_inventors`',
 '`PatentsView_20220630`.`temp_id_mapping_location`',
 '`{{params.x_database}}`.`location_inventor`',
 '`PatentsVie

In [46]:
len(final_query_list)

52

In [322]:
len(all_insert_queries)

115

In [323]:
G=nx.DiGraph()

In [12]:
# FINAL QUERY PULL 
dependencies = []
depend = pd.DataFrame(columns = ['DestinationTable', 'Tables', 'Query', 'LocationTable', 'num_location_id'])
for insert_query in all_insert_queries:
#     if "join" in insert_query:
#         temp = insert_query[:insert_query.index("join") + len("join")]
#     if "group by" in temp:
#         temp = temp[:temp.index("group by") + len("group by")]
    print("----------")
    print(insert_query)
    print("----------")
    loc_id_list = [s for s in insert_query.split(" ") if 'location_id' in s]
    print(loc_id_list)

    if len(loc_id_list)>=1:
        # TABLE & DATABASE WORD 
        insert_destination_search = re.search("INSERT INTO\s+(`[^`]+`\.`[^`]+`)", insert_query, re.IGNORECASE)
        if insert_destination_search is None:
            insert_destination_search = re.search("INSERT IGNORE INTO\s+(`[^`]+`\.`[^`]+`)", insert_query, re.IGNORECASE)
        print(insert_destination_search)
        destination = insert_destination_search.groups(1)[0]
        table_finder = re.compile("`[^`]+`\.`[^`]+`",  re.IGNORECASE)
        tables = table_finder.findall(insert_query)
        loc_table = [s for s in tables if 'location' in s]
        print("----------")
        print(tables)
        print("----------")
#         G.add_node(destination, attr_dict= {
#                 "sql":
#                 re.sub(
#                     "`PatentsView`(\.`[^`]+`)",
#                     "`{{reporting_database}}`\1",
#                     re.sub(
#                         "`patent`(\.`[^`]+`)",
#                         "`{{raw_database}}`\1",
#                         insert_query,
#                         flags=re.IGNORECASE),
#                     flags=re.IGNORECASE)
#             })
        dependencies.append(tables)
#         for t in tables:
#             G.add_edge(destination, t, )
        depend = depend.append({'DestinationTable': destination,'Tables': tables,'LocationTable': loc_table, 'Query' : insert_query , 'num_location_id' : len(loc_id_list)},
        ignore_index = True)

print(depend)


----------
insert into `PatentsView_20220630`.`temp_patent_firstnamed_assignee` ( `patent_id`, `assignee_id`, `persistent_assignee_id`, `location_id`, `persistent_location_id`, `city`, `state`, `country`, `latitude`, `longitude` ) select p.`id`, ta.`new_assignee_id`, ta.`old_assignee_id`, tl.`new_location_id`, tl.`old_location_id_transformed`, nullif(l.`city`, ''), nullif(l.`state`, ''), nullif(l.`country`, ''), l.`latitude`, l.`longitude` from `patent`.`patent` p left outer join `patent`.`rawassignee` ra on ra.`patent_id` = p.`id` and ra.`sequence` = 0 left outer join `PatentsView_20220630`.`temp_id_mapping_assignee` ta on ta.`old_assignee_id` = ra.`assignee_id` left outer join `patent`.`rawlocation` rl on rl.`id` = ra.`rawlocation_id` left outer join `patent`.`location` l on l.`id` = rl.`location_id` left outer join `PatentsView_20220630`.`temp_id_mapping_location` tl on l.`id` = tl.`old_location_id` where (ta.`new_assignee_id` is not null or tl.`new_location_id` is not null) and p.v

In [ ]:
all_insert_queries = []
for sql_file in list1:
    all_sql_content = open(sql_file).read()
    sql_statements = sqlparse.split(all_sql_content)

    for sql_statement in sql_statements:
        query_lines = []
        parsed_statement = sqlparse.parse(sql_statement)[0]
        for sql_line in parsed_statement.value.split("\n"):
            if not sql_line.startswith("#"):
                query_lines.append(" ".join(sql_line.split()))
        if len(query_lines) < 1:
            continue
        single_line_query = " ".join(query_lines).replace("[\s]+", " ").replace(
            "{{params.reporting_database}}", "PatentsView_20220630").replace(
                "{{params.raw_database}}", "patent")
        if parsed_statement.get_type().lower() == 'ddl':
            if single_line_query.startswith("DROP"):
                continue
        if parsed_statement.get_type().lower() == 'insert':
            all_insert_queries.append([single_line_query, sql_file])

In [16]:
final_table_list = []
for couple in dependencies:
    for i in couple:
        if '`patent`.' not in i: 
            final_table_list.append(i)

In [56]:
final_table_list.append('patent_inventor')
final_table_list.append('patent_assignee')

In [58]:
final_query_list = []
for insert_query, sql_file in all_insert_queries:
    for table in final_table_list:
        if table in insert_query:
            final_query_list.append([insert_query, sql_file])

In [59]:
len(final_query_list)

55

In [60]:
# FIND ALL INDEX QUERIES FOR TABLE LIST: 
final = pd.DataFrame(final_query_list, columns = ['query','file'])

In [61]:
final

,query,file
0,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
1,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
2,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
3,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
4,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
5,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
6,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
7,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
8,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql
9,alter table `PatentsView_20220630`.`location` ...,05_10_index.sql


In [52]:
final.to_csv("final.csv")

In [54]:
pwd

'/Users/bcard/Library/CloudStorage/OneDrive-AmericanInstitutesforResearchintheBehavioralSciences/projects/patentsview/PatentsView-DB/reporting_database_generator'